In [ ]:
import pandas as pd
import glob
import numpy as np
import cloudscraper
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import clear_output

manualColumns = ['Background', 
                'Hair Style', 
                'Hair Color', 
                'Facial Hair', 
                'Top', 
                'Bottom',
                'Footwear',
                'Ring',
                'Headwear',
                'Glasses',
                'Piercings', 
                'Accessories']

In [ ]:
projectFiles = ['./Attributes_1_5000.csv',
'./Attributes_5001_10000.csv',
'./Attributes_10001_15000.csv',
'./Attributes_15001_20000.csv',
'./Attributes_20001_25000.csv',
'./Attributes_25001_30000.csv', 
'./Attributes_30001_35000.csv',
'./Attributes_35001_40000.csv',
'./Attributes_40001_45000.csv', 
'./Attributes_45001_50000.csv', 
'./Attributes_50001_55000.csv', 
'./Attributes_55001_60000.csv',
'./Attributes_60001_65000.csv',
'./Attributes_65001_70000.csv',
'./Attributes_70001_75000.csv',
'./Attributes_75001_80000.csv',
'./Attributes_80001_85000.csv',
'./Attributes_85001_90000.csv',
'./Attributes_90001_95000.csv',
'./Attributes_95000_100000.csv']

#df = pd.concat(map(pd.read_csv, glob.glob('./Attributes_1*.csv')))
df = pd.concat(map(pd.read_csv, projectFiles))

In [ ]:
mci = manualColumns.copy()
mci.insert(0,'ID')
df.columns = mci

In [ ]:
validCount = len(df.index) - len(np.where(pd.isnull(df['Background']))[0])
print(validCount)

In [ ]:
missingIDs = np.where(pd.isnull(df['Background']))[0]
print(missingIDs)

In [ ]:
listing = {}

In [ ]:
%%time
startRange = 1
stopRange = 100000

manualColums = ['Background', 
                'Hair Style', 
                'Hair Color', 
                'Facial Hair', 
                'Top', 
                'Bottom',
                'Footwear',
                'Ring',
                'Headwear',
                'Glasses',
                'Piercings', 
                'Accessories']

refStartx = startRange

#for x in range(startRange, stopRange):
for x in (missingIDs):
    scraper = cloudscraper.create_scraper(
        delay=1,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x + 1)).text

    soup = BeautifulSoup(html, 'lxml')

    rows = soup.find_all('div', 'label')

    rowAttributes = {}
    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = "None"
        
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    output_list = [(rowAttributes[key]) for key in (manualColums) if key in rowAttributes]

    
    listing[str(x+1)] = output_list
    clear_output(wait=True)
    print(x+1)
print(listing)
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColums)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv('Attributes_' + str(missingIDs[0]) + '_' + str(missingIDs[len(missingIDs)-1]) + '.csv')
refStartx = x+1
listing.clear()

In [ ]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index')  #pd.DataFrame(listing, orient='index')

In [ ]:
data_dictionary

In [ ]:
for count in range(0, len(manualColums)):
    print(count)
    print(data_dictionary[manualColums[count]].value_counts())
    data_dictionary[manualColums[count]].value_counts(normalize=True)
    print('---------------------------------')

In [ ]:
data_dictionary.to_csv('Attributes_' + str(startRange) + '_' + str(stopRange-1) + '.csv')

In [ ]:
print('done')